In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
% cd '/content/gdrive/MyDrive/Colab Notebooks/university'

/content/gdrive/MyDrive/Colab Notebooks/university


In [ ]:
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_cats.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt

!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_aspects.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_reviews.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_cats.txt

!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_cats.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_reviews.txt
!wget https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt

In [5]:
!pip install --quiet pytextspan
!pip install --quiet natasha
!pip install --quiet transformers
!pip install --quiet datasets
!pip install --quiet pymorphy2

     |████████████████████████████████| 282 kB 5.4 MB/s 
     |████████████████████████████████| 34.4 MB 131 kB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 49 kB 5.3 MB/s 
     |████████████████████████████████| 41 kB 115 kB/s 
     |████████████████████████████████| 8.2 MB 25.0 MB/s 
     |████████████████████████████████| 3.4 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 45.9 MB/s 
     |████████████████████████████████| 3.3 MB 35.2 MB/s 
     |████████████████████████████████| 895 kB 42.4 MB/s 
     |████████████████████████████████| 61 kB 441 kB/s 
     |████████████████████████████████| 306 kB 5.4 MB/s 
     |████████████████████████████████| 132 kB 47.3 MB/s 
     |████████████████████████████████| 243 kB 44.6 MB/s 
     |████████████████████████████████| 1.1 MB 36.2 MB/s 
     |████████████████████████████████| 160 kB 49.6 MB/s 
     |████████████████████████████████| 192 kB 10.7 MB/s 
     |███████████████████

In [6]:
import datasets

import pandas as pd

from sklearn.metrics import classification_report, accuracy_score

import nltk
from nltk import sent_tokenize
import textspan

from tqdm.notebook import tqdm

from pymorphy2.tokenizers import simple_word_tokenize

from razdel import sentenize
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)


import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
nat = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)
segmenter = Segmenter()

In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Предобученная на классификацию сентимента модель

In [33]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment-rusentiment', return_dict=True)
model = model.to(device)

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/495 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/952 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

In [12]:
train_texts = pd.read_csv('train_split_reviews.txt', delimiter='\t', names=['text_id','text'])

dev_texts = pd.read_csv(
    'dev_reviews.txt', 
    delimiter='\t', 
    names=['text_id','text']
)

In [13]:
train_texts.head()

,text_id,text
0,25709,"Были 1 февраля. Я второй раз, подруга - первый..."
1,10099,Были в Дитае на прошлой неделе с подругой. Заш...
2,1105,Один из моих любимых ресторанов. С самого откр...
3,3647,Остались очень довольны. Хотим поблагодарить ш...
4,37070,приехали к друзьям в Питер из Москвы на Новый ...


In [14]:
train_asp = pd.read_csv(
    'train_split_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)

dev_asp = pd.read_csv(
    'dev_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)

dev_pred_asp = pd.read_csv(
    'dev_pred_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)

Для простоты сджойним таблицы, каждому mention соответствует его текст, тексты повторяются

In [27]:
full_df = train_texts.join(train_asp.set_index('text_id'), on='text_id', how='left')
full_df_dev = dev_texts.join(dev_asp.set_index('text_id'), on='text_id', how='left')

In [15]:
full_df_pred = dev_texts.join(dev_pred_asp.set_index('text_id'), on='text_id', how='left')

In [16]:
full_df_pred.head()

,text_id,text,category,mention,start,end,sentiment
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,приняла заказ,254,267,neutral
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,дала меню,208,217,neutral
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,качество обслуживания,671,692,neutral
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,проводила к столу,188,205,neutral
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Whole,ресторан,337,345,neutral


Для каждого выделенного упоминания находим ближайших синтаксических соседей в одном предложении и попробуем предсказывать на них 

In [17]:
def get_sentence(text, ment_start, ment_end):
    #  выделяем то предожение, к которому относится mention
    sents = list(sentenize(text))
    for i, span in enumerate(sents):
        if span.start <= ment_start < ment_end <= span.stop:
            return span.text, span.start

In [18]:
def get_neighbour(doc, ment_start, ment_end):
    dependents = []
    head_id = None
    head_text = ''
    for token in doc.tokens:
        if token.start == ment_start or token.stop == ment_end:
            id_mention = token.id
            head_id = token.head_id
            token_text = token.text 
            break
            
            
    for token in doc.tokens:
        #  записываем текстовый вид вершины
        if token.id == head_id:
            head_text += token.text
        #  записываем прямые зависимые упоминания
        if token.head_id == id_mention:
            dependents.append(token.text)
        #  если у вершины (глагола) есть зависимое "не", то его тоже включаем в соседей
        if token.head_id == head_id and token.text.lower() == 'не':
            head_text = 'не ' + head_text
        
    
    if head_text:
        res = [head_text, token_text] + dependents
    else:
        res = [token_text] + dependents

    return res

In [19]:
def get_nearest_synt_neighbour(row):
    text = row['text']
    mention = row['mention']
    ment_start = row['start']
    ment_end = row['end']
    sentence, offset = get_sentence(text, ment_start, ment_end)

    doc = Doc(sentence)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.parse_syntax(syntax_parser)

    neigh = get_neighbour(doc, ment_start - offset, ment_end - offset)
    
    return neigh

In [28]:
full_df['extended_context'] = full_df.apply(lambda x: get_nearest_synt_neighbour(x), axis=1)
full_df_dev['extended_context'] = full_df_dev.apply(lambda x: get_nearest_synt_neighbour(x), axis=1)

In [20]:
full_df_pred['extended_context'] = full_df_pred.apply(lambda x: get_nearest_synt_neighbour(x), axis=1)

In [211]:
full_df.head()

,text_id,text,category,mention,start,end,sentiment,extended_context
0,25709,"Были 1 февраля. Я второй раз, подруга - первый...",Service,девушки,168,175,positive,"[спросили, девушки]"
0,25709,"Были 1 февраля. Я второй раз, подруга - первый...",Whole,заведения,302,311,neutral,"[прелестях, заведения, данного]"
0,25709,"Были 1 февраля. Я второй раз, подруга - первый...",Interior,диваны,347,353,positive,"[диваны, Мягкие, россыпью, .]"
0,25709,"Были 1 февраля. Я второй раз, подруга - первый...",Interior,подушек,365,372,positive,"[россыпью, подушек]"
0,25709,"Были 1 февраля. Я второй раз, подруга - первый...",Interior,музыка,402,408,positive,"[музыка, приятная, негромкая, полумрак, .]"


Для валидации методов далее будем пытаться предсказывать сентимент для gold standard аспектов на тесте

Делаем предсказания, где X - это токены аспекта объединенные с его ближайшими синтаксическими соседями, y - сентимент 

In [29]:
mapper = {'neutral': 0, 'positive': 1, 'negative': 2, 'both': 0}

extracted = list(full_df_dev.extended_context.values)
y_true = [mapper[label] for label in full_df_dev.sentiment.values]

In [160]:
extracted[:5]

[['аппетит', '"'],
 ['то', 'встретил', 'случайно', 'нас', 'время'],
 ['девушка', 'менеджер-'],
 ['полон', 'девушка', 'менеджер-', 'темноволосая', 'стройная'],
 ['пришла', 'проводила', 'столу']]

In [144]:
@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted

In [145]:
y_pred = [predict(' '.join(text)) for text in extracted]

In [146]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.23      0.96      0.37       216
           1       0.95      0.31      0.46       795
           2       0.80      0.07      0.12       179

    accuracy                           0.39      1190
   macro avg       0.66      0.44      0.32      1190
weighted avg       0.80      0.39      0.39      1190



Качество невысокое. Попробуем использовать не только ближайших соседей, а все предложение - с допущением, что в одном предложении редко встречаются противоположные мнения для аспектов.

In [21]:
def get_sent(row):
    text = row['text']
    mention = row['mention']
    ment_start = row['start']
    ment_end = row['end']
    sentence, _ = get_sentence(text, ment_start, ment_end)
    
    return sentence

In [ ]:
full_df['sentence'] = full_df.apply(get_sent, axis=1)
full_df_dev['sentence'] = full_df_dev.apply(get_sent, axis=1)

In [23]:
full_df_pred['sentence'] = full_df_pred.apply(get_sent, axis=1)

In [148]:
y_pred_2 = [predict(sentence) for sentence in full_df_dev.sentence]

In [149]:
print(classification_report(y_true, y_pred_2))

              precision    recall  f1-score   support

           0       0.37      0.73      0.49       216
           1       0.88      0.75      0.81       795
           2       0.71      0.32      0.44       179

    accuracy                           0.68      1190
   macro avg       0.65      0.60      0.58      1190
weighted avg       0.76      0.68      0.70      1190



Классификация сентимента отзывов узкой темы может иметь свои специфические особенности - попробуем дообучить модель на наших трейновых данных (с тем же допущением, что аспект описывает целое предложение)

In [142]:
full_df.head()

,text_id,text,category,mention,start,end,sentiment,extended_context,sentence
0,25709,"Были 1 февраля. Я второй раз, подруга - первый...",Service,девушки,168,175,positive,"[спросили, девушки]",На входе девушки спросили о наших предпочтения...
0,25709,"Были 1 февраля. Я второй раз, подруга - первый...",Whole,заведения,302,311,neutral,"[прелестях, заведения, данного]",Предварительно очень подробно рассказав обо вс...
0,25709,"Были 1 февраля. Я второй раз, подруга - первый...",Interior,диваны,347,353,positive,"[диваны, Мягкие, россыпью, .]",Мягкие диваны с россыпью подушек.
0,25709,"Были 1 февраля. Я второй раз, подруга - первый...",Interior,подушек,365,372,positive,"[россыпью, подушек]",Мягкие диваны с россыпью подушек.
0,25709,"Были 1 февраля. Я второй раз, подруга - первый...",Interior,музыка,402,408,positive,"[музыка, приятная, негромкая, полумрак, .]",Довольно приятная негромкая музыка и легкий по...


In [ ]:
train_data = [{'label': mapper[label], 'text': text} for text, label in zip(full_df.sentence.values.tolist(), full_df.sentiment.values.tolist())]
test_data = [{'label': mapper[label], 'text': text} for text, label in zip(full_df_dev.sentence.values.tolist(), full_df_dev.sentiment.values.tolist())]

In [ ]:
df_train = pd.DataFrame(train_data)
df_test = pd.DataFrame(test_data)

In [ ]:
dataset_train = datasets.Dataset.from_pandas(df_train)
dataset_test = datasets.Dataset.from_pandas(df_test)

In [ ]:
raw_datasets = datasets.DatasetDict({"train":dataset_train, "test":dataset_test})

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model, args=training_args, train_dataset=full_train_dataset, eval_dataset=full_eval_dataset
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 3573
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1341


Step,Training Loss
500,0.721200
1000,0.569100


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1341, training_loss=0.6045681400853782, metrics={'train_runtime': 2619.0397, 'train_samples_per_second': 4.093, 'train_steps_per_second': 0.512, 'total_flos': 2820312724626432.0, 'train_loss': 0.6045681400853782, 'epoch': 3.0})

In [ ]:
model.save_pretrained('finetuned')

Configuration saved in fine_tuned/config.json
Model weights saved in fine_tuned/pytorch_model.bin


Предсказание на зафайнтьюненной модели

In [24]:
model_new = AutoModelForSequenceClassification.from_pretrained('./finetuned', return_dict=True)
model_new = model_new.to(device)

In [31]:
@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = model_new(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted

In [ ]:
y_pred = [predict(' '.join(text)) for text in extracted]

In [153]:
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.34      0.70      0.45       216
           1       0.85      0.67      0.75       795
           2       0.56      0.34      0.42       179

    accuracy                           0.63      1190
   macro avg       0.58      0.57      0.54      1190
weighted avg       0.71      0.63      0.65      1190



In [154]:
y_pred_2 = [predict(sentence) for sentence in full_df_dev.sentence]

In [155]:
print(classification_report(y_true, y_pred_2))

              precision    recall  f1-score   support

           0       0.56      0.47      0.51       216
           1       0.84      0.91      0.87       795
           2       0.66      0.53      0.59       179

    accuracy                           0.77      1190
   macro avg       0.68      0.64      0.66      1190
weighted avg       0.76      0.77      0.76      1190



Дообучение помогло, лучшие результаты предсказаний оказываются на полных предложениях. Сделаем предсказания на наших выделенных данных и сохраним в файл (отдельно сохраним уверенность модели для каждого предсказания, чтобы использовать это в третьем задании)

In [35]:
@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt').to(device)
    outputs = model_new(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    conf = torch.max(predicted, dim=1)
    predicted = torch.argmax(predicted, dim=1).cpu().numpy()
    return predicted, conf

In [36]:
full_df_pred.head()

,text_id,text,category,mention,start,end,sentiment,extended_context,sentence
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,приняла заказ,254,267,neutral,"[полон, приняла, заказ]","Зашли в""аппетит"" случайно.Не смотря на то,что ..."
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,дала меню,208,217,neutral,"[пришла, дала, и, меню]","Зашли в""аппетит"" случайно.Не смотря на то,что ..."
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,качество обслуживания,671,692,neutral,"[место, качество, и, высокое, обслуживания]","В общем очень хорошее место, разнообразное вку..."
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,проводила к столу,188,205,neutral,"[пришла, проводила, столу]","Зашли в""аппетит"" случайно.Не смотря на то,что ..."
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Whole,ресторан,337,345,neutral,"[полон, ресторан]","Зашли в""аппетит"" случайно.Не смотря на то,что ..."


In [37]:
mapper

{'both': 0, 'negative': 2, 'neutral': 0, 'positive': 1}

In [38]:
#  замапим обратно индексы в текстовые названия sentiment
ind2text = {0: 'neutral', 1: 'positive', 2: 'negative'}

In [39]:
y_pred_3 = [predict(sentence) for sentence in full_df_pred.sentence]

In [40]:
pred_labels = [ind2text[label[0][0]] for label in y_pred_3]

In [41]:
y_pred_3[0][1][0].item()

0.6505323648452759

In [42]:
pred_confs = [label[1][0].item() for label in y_pred_3]

In [43]:
len(pred_confs) == len(pred_labels) == len(full_df_pred)

True

In [44]:
full_df_pred['sentiment'] = pred_labels

In [45]:
full_df_pred.head(2)

,text_id,text,category,mention,start,end,sentiment,extended_context,sentence
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,приняла заказ,254,267,positive,"[полон, приняла, заказ]","Зашли в""аппетит"" случайно.Не смотря на то,что ..."
0,13823,"Зашли в""аппетит"" случайно.Не смотря на то,что ...",Service,дала меню,208,217,positive,"[пришла, дала, и, меню]","Зашли в""аппетит"" случайно.Не смотря на то,что ..."


In [46]:
import pickle

with open('confs.pkl', 'wb') as file:
  pickle.dump(pred_confs, file)

In [47]:
with open('dev_pred_aspects.txt', 'w') as f:
    for text_id, category, mention, start, end, sentiment in zip(full_df_pred.text_id,
                                                                full_df_pred.category,
                                                                full_df_pred.mention.values, 
                                                                full_df_pred.start.values, 
                                                                full_df_pred.end.values,
                                                                full_df_pred.sentiment.values,
                                                                ):
      print(text_id, category, mention, start, end, sentiment, sep="\t", file=f)